# Transfer learning and Mechanisms of Action
For the last few weeks, I have been thinking the following: This data set cries of transfer learning. Why?
For the following reasons:
* We have an additional dataset with labels
* Most people here tend to think this is a multioutput scenario. If this is true, inner layers should not depend too much on outputs
* Some of the outputs have only a few examples. In two cases, there is a single positive outcome (therefore no way to predict it?). Since we have apparently very incomplete information, I would think more the more information the better.

All this smells like a textbook example of transfer learning, doesn't it? Yet after weeks of trying, I have never achieved better outcome with transfer model than with the same architecture which was directly fitted (not much worse, sure... but on competition score, I never scored better with transfer model). 

Any idea why?

* Is my thinking wrong all along for some apparent reason?
* Is my model construction erroneous?
* What is going on here?

This only got public score of 0.23. This is not a particularly optimized version of the notebook... but still what's wrong with transfer learning here?

## Display input data

## Read data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Training data

In [ ]:
train_feat_df = pd.read_csv("/kaggle/input/lish-moa/train_features.csv", index_col='sig_id')
train_feat_df.loc[:,'cp_dose_cat'] = train_feat_df.cp_dose.astype('category').cat.codes
train_feat_df.loc[:,'cp_type_cat'] = train_feat_df.cp_type.astype('category').cat.codes
train_feat_df.loc[:,'cp_time_cat'] = train_feat_df.cp_time.astype('category').cat.codes
train_feat_df.drop(columns=['cp_dose', 'cp_type' ], inplace=True)
train_feat_df.drop(columns=['cp_time' ], inplace=True)
train_tg_scored_df = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv", index_col='sig_id')
train_tg_nonscored_df = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv", index_col='sig_id')


In [ ]:
train_feat_df = train_feat_df.loc[train_feat_df.cp_type_cat==1, train_feat_df.columns != "cp_type_cat"]
train_feat_df.head(2)

### Test data

In [ ]:
test_feat_df = pd.read_csv("/kaggle/input/lish-moa/test_features.csv", index_col='sig_id')
test_feat_df.loc[:,'cp_dose_cat'] = test_feat_df.cp_dose.astype('category').cat.codes
test_feat_df.loc[:,'cp_type_cat'] = test_feat_df.cp_type.astype('category').cat.codes
# ??
test_feat_df.loc[:,'cp_time_cat'] = test_feat_df.cp_time.astype('category').cat.codes
test_feat_df.drop(columns=['cp_dose', 'cp_type' ], inplace=True)
test_feat_df.drop(columns=['cp_time' ], inplace=True)
control_indexes_test = [test_feat_df.index.get_loc(x) for x in test_feat_df[test_feat_df.cp_type_cat==0].index.values]


In [ ]:
#test_feat_df = test_feat_df.loc[test_feat_df.cp_type_cat==1, test_feat_df.columns != "cp_type_cat"]
test_feat_df = test_feat_df.loc[:, test_feat_df.columns != "cp_type_cat"]
test_feat_df.head(2)

## Stratified train test split for multioutput case

In [ ]:
from skmultilearn.model_selection import IterativeStratification
# sample_distribution_per_fold = [test_size, 1.0-test_size]
stratifier = IterativeStratification(n_splits=2, order=2, sample_distribution_per_fold=[0.3, 0.7])
train_indexes, test_indexes = next(stratifier.split(train_feat_df, train_tg_scored_df.loc[train_feat_df.index,:]))
X_train, Y_train = train_feat_df.iloc[train_indexes], train_tg_scored_df.iloc[train_indexes]
X_val, Y_val = train_feat_df.iloc[test_indexes], train_tg_scored_df.iloc[test_indexes]

## Build keras transfer learning model
### First create the featurizer which uses nonscored data

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer, BatchNormalization
from tensorflow.keras.layers import Dropout

In [ ]:
model_nonscored = Sequential()
model_nonscored.add(BatchNormalization(input_shape=(X_train.shape[1],)))
model_nonscored.add(Dense(2048, activation='relu', input_shape=(875,)))
model_nonscored.add(BatchNormalization())
model_nonscored.add(Dense(1024, activation='relu'))
model_nonscored.add(BatchNormalization())
model_nonscored.add(Dense(512, activation='relu'))
model_nonscored.add(Dropout(0.2))
model_nonscored.add(BatchNormalization(name="Last_batch_norm_l"))
model_nonscored.add(Dense(402, activation='sigmoid'))

In [ ]:
p_min = 0.001
p_max = 0.999

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -tf.keras.backend.mean(y_true*tf.keras.backend.log(y_pred) + (1-y_true)*tf.keras.backend.log(1-y_pred))

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_logloss', factor=0.1, verbose=0,mode='min',
                              patience=3, min_lr=1E-7)
early_st = tf.keras.callbacks.EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=7, verbose=0, mode='min',
    baseline=None, restore_best_weights=True)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# default learning rate = 0.001
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False
)

model_nonscored.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001), metrics=logloss )
model_nonscored.summary()

In [ ]:
tf.keras.utils.plot_model(model_nonscored, to_file='model.png')

In [ ]:
_ = model_nonscored.fit(train_feat_df, train_tg_nonscored_df.loc[train_feat_df.index,:], epochs=10, callbacks=[reduce_lr, early_st])

This might be an issue. Here I did not split the dataset and used the whole training set to fit the transfer model. Yet I also tested the proper split and it did not result in anything better...

In [ ]:
from tensorflow.keras.models import Model

I know this is not the most elegant usage of functional API. But it seems to work...

In [ ]:
model1_out = model_nonscored.get_layer('Last_batch_norm_l').output
out = Dense(206, activation='sigmoid', name='output_layer_tlmodel')(model1_out)
tlmodel = Model(inputs = model_nonscored.inputs, outputs = out)

In [ ]:
#last_frozen = "dropout_1"
last_frozen = "Last_batch_norm_l"
for layer in tlmodel.layers:
    layer.trainable = False    
    if layer.name == last_frozen:
        break
for layer in tlmodel.layers:
    print(f"{layer.name}: {layer.trainable}.")

In [ ]:
tlmodel.summary()

In [ ]:
tlmodel.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001), metrics=logloss )

In [ ]:
_ = tlmodel.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=1000, callbacks=[reduce_lr, early_st])

In [ ]:
# clipping predictions
p_min = 0.001
p_max = 0.999


In [ ]:
predictions = np.clip(tlmodel.predict(test_feat_df),p_min,p_max)
predictions[control_indexes_test, :] = 0

predictions_df = pd.DataFrame(predictions)
predictions_df.shape

In [ ]:
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv', index_col='sig_id')
print(submission.shape)
submission.head(2)

In [ ]:
predictions_df.index = submission.index

In [ ]:
predictions_df.head(3)

In [ ]:
predictions_df.columns = submission.columns
predictions_df.head(3)

In [ ]:
predictions_df.to_csv("submission.csv")